In [1]:
import os
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [ ]:
# Import the required LangChain modules
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

In [34]:
# Define LLM to be used
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

In [33]:
# Alternate LLMs

from langchain_groq import ChatGroq

#llm=ChatGroq(model_name="Gemma2-9b-It")

#llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")


In [4]:
# Define the Embeddings model

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\91852\Documents\Machine Learning\Gen AI\AI_Agents\env_langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Step 1 - Document Ingestion

In [6]:
### Read the Script

from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('Scripts/Dummy_Script.pdf')

docs = loader.load()

Step 2 - Document Splitting

In [8]:
# Split the document and create Chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = text_splitter.split_documents(docs)

Step 3 - Indexing (Embedding Generation and Storing in Vector Store)

In [9]:
# Create the Vector Store

vector_store = FAISS.from_documents(chunks, embeddings)

Step 4 - Retrieval

In [10]:
# Create a Retriever

retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs={'k':4})

Step 5 - Augmentation

In [11]:
# Create a Prompt Template

prompt = PromptTemplate(
    template="""
      You are a SAS code reviewer focussing on Data Lineage.
      Answer ONLY from the provided transcript context.
      Give your answer in 2 -3 sentences.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [32]:
# Test the Prompt

question          = "What is the source table?"
retrieved_docs    = retriever.invoke(question)

context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

final_prompt = prompt.invoke({"context": context_text, "question": question})

answer = llm.invoke(final_prompt)
print(answer.content)

The source table for the `processed.customer_orders_raw` dataset is `rawdata.orders`. The `processed.final_customer_data` dataset is merged from `processed.customer_sales_summary` and `rawdata.customer_demographics`.


Step 6 - Building a Chain

In [13]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [14]:
def format_docs(retrieved_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [15]:
# Parallel Chain

parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
    }
)

In [28]:
# Final Chain

parser = StrOutputParser()

main_chain = parallel_chain | prompt | llm | parser

In [17]:
question1 = "What child tables are created from the table 'customer_orders_filtered' ?"

In [18]:
main_chain.invoke(question1)

'Based on the provided code, the table `processed.customer_sales_summary` is created from `processed.customer_orders_filtered`. This child table is the result of a `proc sql` step that aggregates sales and counts orders by customer.'

In [19]:
question2 = "What all will break if 'customer_orders_filtered' is deleted ?"

In [30]:
main_chain.invoke(question2)

'If `processed.customer_orders_filtered` is deleted, the `proc sql` step that creates `processed.customer_sales_summary` will fail. This is because `processed.customer_sales_summary` directly uses `processed.customer_orders_filtered` as its input table. Consequently, the subsequent `data processed.final_customer_data` step would also break as it relies on `processed.customer_sales_summary` for its data.'

In [31]:
question3 = "What will break if column 'quantity' is deleted from the table 'orders' ?"

In [29]:
main_chain.invoke(question3)

"If the 'Quantity' column is deleted from the `rawdata.orders` table, the `OrderValue` variable in the `processed.customer_orders_raw` dataset will break. This is because `OrderValue` is calculated as `Quantity * UnitPrice`."